# ETL Processes
***
## Lists of tasks:
- ### <del>Consolidating Datasets</del>
- ### Normalising/Restructuring Tables
- ### Exploratory Data Analysis
- ### Data Cleaning
- ### Package ETL.py into a Class
***

## Content:
- ### [Consumer Dataset](#Consumer-dataset)
- ### [Transaction Dataset](#Transaction-dataset)
- ### [Merchant Dataset](#Merchant-dataset)
- ### [Data Aggregations](#Aggregation)


In [1]:
import pandas as pd
import numpy as np
import os
import re

# Set working directory
if not "/data/tables" in os.getcwd():
    os.chdir("../data/tables")

from pyspark.sql import SparkSession
from pyspark.shell import spark
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
import matplotlib.pyplot as plt

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.7.4 (default, Aug 13 2019 15:17:50)
Spark context Web UI available at http://169.254.224.157:4040
Spark context available as 'sc' (master = local[*], app id = local-1661970285241).
SparkSession available as 'spark'.


# Consumer dataset

In [2]:
# Read csv file
consumer = spark.read.option("delimiter", "|").csv('tbl_consumer.csv', header = True)
consumer

name,address,state,postcode,gender,consumer_id
Yolanda Williams,413 Haney Gardens...,WA,6935,Female,1195503
Mary Smith,3764 Amber Oval,NSW,2782,Female,179208
Jill Jones MD,40693 Henry Greens,NT,862,Female,1194530
Lindsay Jimenez,00653 Davenport C...,NSW,2780,Female,154128
Rebecca Blanchard,9271 Michael Mano...,WA,6355,Female,712975
Karen Chapman,2706 Stewart Oval...,NSW,2033,Female,407340
Andrea Jones,122 Brandon Cliff,QLD,4606,Female,511685
Stephen Williams,6804 Wright Crest...,WA,6056,Male,448088
Stephanie Reyes,5813 Denise Land ...,NSW,2482,Female,650435
Jillian Gonzales,461 Ryan Common S...,VIC,3220,Female,1058499


In [3]:
print(f"Dataset details: \n\tNumber of rows: {consumer.count()}", \
      f"\n\tNumber of distinct Consumer ID: {consumer.select('consumer_id').distinct().count()}", \
      f"\n\tNumber of distinct Postcodes: {consumer.select('postcode').distinct().count()}")

Dataset details: 
	Number of rows: 499999 
	Number of distinct Consumer ID: 499999 
	Number of distinct Postcodes: 3167


Note: 
- The **address field is fake** and derived from USA street names. We have included it to mimic a more realistic dataset, but the streets themselves are non-existent and if there are any matches, it will be a pure coincidence. <font color='red'>**Not sure what sort of information we can extract here if they are all fake</font> 
- The **postcode field is accurate** and should be **used for aggregated analysis** for joining with other geospatial datasets for demographic information (i.e ABS datasets) <font color='red'>**Highly relevant for geospatial analysis</font> 
- There is roughly a **uniform distribution at the state level** (i.e number of consumers per state is the same for all states).

### Checking for missing values in consumer dataset

In [107]:
def missing_values_check(sdf):
    """Check missing values in each column of the spark dataframe"""
    
    missing_count = sdf.select([count(when(col(c).contains('None') | \
                                           col(c).contains('NULL') | \
                                          (col(c) == '' ) | \
                                           col(c).isNull() | \
                                           isnan(c), c))
                                     .alias(c)
                                for c, dtype in sdf.dtypes if dtype != 'date'])
    
    return missing_count


In [109]:
missing_values_check(consumer)

name,address,state,postcode,gender,consumer_id
0,0,0,0,0,0


### User detail dataset

In [4]:
user_detail = spark.read.parquet("consumer_user_details.parquet")
user_detail

user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975
6,407340
7,511685
8,448088
9,650435
10,1058499


In [5]:
print(f"Dataset details: \n\tNumber of rows: {user_detail.count()}", \
      f"\n\tNumber of distinct User ID: {user_detail.select('user_id').distinct().count()}", \
      f"\n\tNumber of distinct Consumer ID: {user_detail.select('consumer_id').distinct().count()}")

Dataset details: 
	Number of rows: 499999 
	Number of distinct User ID: 499999 
	Number of distinct Consumer ID: 499999


### Checking for missing values in user detail dataset

In [108]:
missing_values_check(user_detail)

user_id,consumer_id
0,0


Note:
- Due to a difference between the internal system and a poor design choice (for some reason), the transaction tables use a **surrogate key** for each new user_id. <font color='red'>**Transaction dataset uses `user_id` to map customer but customer data are mapped to their own unique `customer_id` so the user detail data serves to map those two together</font> 
- However, the Consumer table has a **unique ID (some are missing on purpose)** field which will require some form of mapping between consumer_id to user_id. <font color='red'>**Might require further investigation and decide on whether it is appropriate to remove</font> 
- An additional mapping table has been provided to join the two datasets together.


# Transaction dataset

In [6]:
transaction = spark.read.parquet("transactions_20210228_20210827_snapshot/")

<div class="alert alert-block alert-warning">
<b>Note:</b> Use the following code to load the transactions files if you have problems running the code above.
</div>

In [10]:
path = "transactions_20210228_20210827_snapshot/"
list_files = os.listdir(path)
list_files = list_files[1:(len(list_files)-1)]
list_files

['order_datetime=2021-08-15',
 'order_datetime=2021-08-23',
 'order_datetime=2021-08-24',
 'order_datetime=2021-04-20',
 'order_datetime=2021-07-07',
 'order_datetime=2021-03-08',
 'order_datetime=2021-04-27',
 'order_datetime=2021-03-30',
 'order_datetime=2021-04-18',
 'order_datetime=2021-04-11',
 'order_datetime=2021-03-06',
 'order_datetime=2021-07-09',
 'order_datetime=2021-03-01',
 'order_datetime=2021-04-16',
 'order_datetime=2021-07-31',
 'order_datetime=2021-04-29',
 'order_datetime=2021-08-25',
 'order_datetime=2021-08-22',
 'order_datetime=2021-08-14',
 'order_datetime=2021-08-13',
 'order_datetime=2021-07-30',
 'order_datetime=2021-04-17',
 'order_datetime=2021-04-28',
 'order_datetime=2021-03-07',
 'order_datetime=2021-04-10',
 'order_datetime=2021-07-08',
 'order_datetime=2021-03-31',
 'order_datetime=2021-04-26',
 'order_datetime=2021-07-01',
 'order_datetime=2021-04-19',
 'order_datetime=2021-07-06',
 'order_datetime=2021-04-21',
 'order_datetime=2021-03-09',
 '.DS_Stor

In [ ]:
# import modules
from pyspark.sql import SparkSession
import functools
 
# explicit function
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

# insert datetime
file_name = os.listdir(path+ list_files[0])[1]
transaction = spark.read.parquet(path+ list_files[0] +"/" + file_name)
for i in list_files[1:]:
    file_name = os.listdir(path + i)[1]
    tmp = spark.read.parquet(path+ i +"/" + file_name)
    transaction = unionAll([transaction, tmp] )

### Inspecting transaction dataset

In [7]:
transaction

user_id,merchant_abn,dollar_value,order_id,order_datetime
18478,62191208634,63.255848959735246,949a63c8-29f7-4ab...,2021-08-20
2,15549624934,130.3505283105634,6a84c3cf-612a-457...,2021-08-20
18479,64403598239,120.15860593212783,b10dcc33-e53f-425...,2021-08-20
3,60956456424,136.6785200286976,0f09c5a5-784e-447...,2021-08-20
18479,94493496784,72.96316578355305,f6c78c1a-4600-4c5...,2021-08-20
3,76819856970,448.529684285612,5ace6a24-cdf0-4aa...,2021-08-20
18479,67609108741,86.4040605836911,d0e180f0-cb06-42a...,2021-08-20
3,34096466752,301.5793450525113,6fb1ff48-24bb-4f9...,2021-08-20
18482,70501974849,68.75486276223054,8505fb33-b69a-412...,2021-08-20
4,49891706470,48.89796461900801,ed11e477-b09f-4ae...,2021-08-20


In [8]:
transaction.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [9]:
min_date, max_date = transaction.select(min("order_datetime"), max("order_datetime")).first()

print(f"Dataset details: \n\tNumber of rows: {transaction.count()}", \
      f"\n\tNumber of distinct order: {transaction.select('order_id').distinct().count()}", \
      f"\n\tPeriod: {min_date} - {max_date}")

Dataset details: 
	Number of rows: 3643266 
	Number of distinct order: 3643266 
	Period: 2021-02-28 - 2021-08-27


### Checking for missing values in transaction dataset

In [110]:
missing_values_check(transaction)

user_id,merchant_abn,dollar_value,order_id
0,0,0,0


In [ ]:
transaction.count()

# Merchant dataset

In [112]:
merchant = spark.read.parquet("tbl_merchants.parquet")
merchant

name,tags,merchant_abn
Felis Limited,"((furniture, home...",10023283211
Arcu Ac Orci Corp...,"([cable, satellit...",10142254217
Nunc Sed Company,"([jewelry, watch,...",10165489824
Ultricies Digniss...,"([wAtch, clock, a...",10187291046
Enim Condimentum PC,([music shops - m...,10192359162
Fusce Company,"[(gift, card, nov...",10206519221
Aliquam Enim Inco...,"[(computers, comP...",10255988167
Ipsum Primis Ltd,"[[watch, clock, a...",10264435225
Pede Ultrices Ind...,([computer progra...,10279061213
Nunc Inc.,"[(furniture, home...",10323485998


In [113]:
merchant.printSchema()

root
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: long (nullable = true)



In [114]:
print(f"Dataset details: \n\tNumber of rows: {merchant.count()}", \
      f"\n\tNumber of distinct Merchant ABN: {merchant.select('merchant_abn').distinct().count()}")

Dataset details: 
	Number of rows: 4026 
	Number of distinct Merchant ABN: 4026


### Checking for missing values in merchant dataset

In [115]:
missing_values_check(merchant)

name,tags,merchant_abn
0,0,0


### The tags column consists of tags, revenue levels and take rate of a merchant
- **Revenue Levels**: (a, b, c, d, e) represents the **level of revenue bands** (unknown to groups). a denotes the smallest band whilst e denotes the highest revenue band. <font color='red'>**Highly relevant in ranking merchant</font> 
- **Take Rate**: the **fee charged by the BNPL firm** to a merchant on a transaction. That is, for each transaction made, a certain percentage is taken by the BNPL firm.<font color='red'>**Highly relevant in ranking merchant</font> 
- The dataset has been created to mimic a Salesforce data extract (i.e salespeople will type in tags and segments within a **free-text** field). <font color='red'>This suggests use of lemmatizating/stemming/fuzzy methods to group similar texts?</font> 
- As such, please be aware of small **human errors** when parsing the dataset.
- For Example, the tag field may have errors as they were manually input by employees.

Since the data is small, we will be using Pandas to deal with Merchant data for convenience.

In [122]:
merchant = merchant.toPandas()

In [123]:
# Display first 5 rows for "tags"
for idx, row in merchant.head(5).iterrows():
    print(row['tags'])


((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))
([cable, satellite, and otHer pay television and radio services], [b], [take rate: 4.22])
([jewelry, watch, clock, and silverware shops], [b], [take rate: 4.40])
([wAtch, clock, and jewelry repair shops], [b], [take rate: 3.29])
([music shops - musical instruments, pianos, and sheet music], [a], [take rate: 6.33])


### Extract Revenue Levels and Take Rate columns

In [124]:
# Function to extract tags, revenue level and take rate from tags column
def extract_tags(arr, category='tags'):
    
    # Split tags into the three components
    arr = arr[1:-1]
    split_arr = re.split('\), \(|\], \[', arr.strip('[()]'))
    
    if category == 'take_rate':
        return re.findall('[\d\.\d]+', split_arr[2])[0]
    
    elif category == 'revenue_level':
        return split_arr[1].lower()
    
    return split_arr[0].lower()


In [125]:
# Extract all three components in tags as standalone columns
merchant['take_rate'] = merchant['tags'].apply(lambda x : extract_tags(x, 'take_rate'))
merchant['revenue_level'] = merchant['tags'].apply(lambda x : extract_tags(x, 'revenue_level'))
merchant['tags'] = merchant['tags'].apply(lambda x : extract_tags(x, 'tags'))


In [126]:
# Check if we extracted the take_rate and rev_level values correctly
print(f"Unique value in Revenue Level: {merchant['revenue_level'].unique()}")
print(f"Range of Take Rate: {merchant['take_rate'].min()} - {merchant['take_rate'].max()}")


Unique value in Revenue Level: ['e' 'b' 'a' 'c' 'd']
Range of Take Rate: 0.10 - 7.00


In [127]:
# Check data type for columns
merchant.dtypes

name             object
tags             object
merchant_abn      int64
take_rate        object
revenue_level    object
dtype: object

In [128]:
merchant['take_rate'] = pd.to_numeric(merchant['take_rate'])

In [129]:
# Check data type for columns
merchant.dtypes

name              object
tags              object
merchant_abn       int64
take_rate        float64
revenue_level     object
dtype: object

In [130]:
merchant.to_csv('../curated/clean_merchant.csv')

# Aggregation

Here we generate various aggregate data to supplement our analyses and modelling.

### Merchant Sales

In [16]:
# Generate data which summarizes merchants' sales
merchant_sales = (transaction.groupby('merchant_abn', 'order_datetime')
                             .agg({'dollar_value':'sum',
                                   'order_id':'count'})
                             .withColumnRenamed('sum(dollar_value)', 'sales_revenue')
                             .withColumnRenamed('count(order_id)', 'no_orders'))

In [17]:
merchant_sales

merchant_abn,order_datetime,sales_revenue,no_orders
90173050473,2021-08-20,10065.92598002776,41
91455531890,2021-08-20,1486.8427457787936,3
14480530534,2021-08-20,3273.1834477999028,2
54062916822,2021-08-20,33.80538233308192,1
73225085327,2021-08-20,342.0587279527964,1
34558201737,2021-08-20,623.7741963160896,13
63523606379,2021-08-20,748.1316659657341,2
15130143460,2021-08-20,3389.0731959179543,6
49167531725,2021-08-20,404.49218244684687,8
10023283211,2021-08-20,957.3137092766215,5


In [117]:
# Download data
merchant_sales.write.parquet("../curated/merchant_sales.parquet")

### Customers Purchase Behaviour

In [18]:
# Generate data which summarizes customers spendings
customer_purchases = (transaction.groupby('user_id', 'order_datetime')
                                 .agg({'dollar_value':'sum',
                                       'order_id':'count'})
                                 .withColumnRenamed('sum(dollar_value)', 'dollar_spent')
                                 .withColumnRenamed('count(order_id)', 'no_orders'))


In [19]:
customer_purchases

user_id,order_datetime,dollar_spent,no_orders
18488,2021-08-20,468.9797223930378,3
686,2021-08-20,231.87884410751695,2
19292,2021-08-20,961.7931435852486,1
778,2021-08-20,64.44320007323647,2
786,2021-08-20,80.29337626952264,1
19476,2021-08-20,506.69827263377283,2
19497,2021-08-20,185.18180257359967,2
19628,2021-08-20,421.706928005088,2
19631,2021-08-20,15.836078616105175,1
19672,2021-08-20,58.311520840379146,3


In [ ]:
# Download data
customer_purchases.write.parquet("../curated/customer_purchase_behaviour.parquet")

### Sales by Region

In [43]:
# Join transaction data with customer data
customer_transaction = (transaction.join(user_detail, transaction.user_id == user_detail.user_id)
                                   .drop(user_detail.user_id))

customer_transaction = (customer_transaction.join(consumer, customer_transaction.consumer_id == consumer.consumer_id)
                                            .drop(consumer.consumer_id)
                                            .select(transaction['*'], consumer.postcode, consumer.state, consumer.gender))

In [44]:
customer_transaction

user_id,merchant_abn,dollar_value,order_id,order_datetime,postcode,state,gender
5630,60956456424,145.26081329000152,1e14adeb-8e13-44f...,2021-08-21,841,NT,Undisclosed
5630,48534649627,120.25889985200416,08476339-f383-4ab...,2021-08-15,841,NT,Undisclosed
5630,60956456424,135.5412540082104,aacfd47a-438b-47f...,2021-08-15,841,NT,Undisclosed
5630,89932674734,95.37693966478514,6d5790c9-0eef-453...,2021-08-16,841,NT,Undisclosed
5630,14089706307,440.12097711482835,43d1361a-1101-41a...,2021-08-16,841,NT,Undisclosed
5630,33604812025,268.4088838875397,4d8b6deb-a4d7-412...,2021-08-13,841,NT,Undisclosed
5630,81219314324,48.20692954489542,6454fbb0-7cbd-4db...,2021-07-23,841,NT,Undisclosed
5630,94472466107,141.26779955902157,d85af812-4f34-46b...,2021-05-21,841,NT,Undisclosed
5630,15903176024,44.68660714566651,4bae0c2f-0d25-4f6...,2021-05-21,841,NT,Undisclosed
5630,73527950332,654.6729455135422,44f25667-6711-4c5...,2021-07-30,841,NT,Undisclosed


In [ ]:
# Download data
customer_transaction.write.parquet("../curated/customer_join_transaction.parquet")


In [45]:
# Aggregate by state -> postcode -> date
sales_by_region = (customer_transaction.groupby('state', 'postcode', 'order_datetime')
                                       .agg({'dollar_value':'sum',
                                             'order_id':'count'})
                                       .withColumnRenamed('sum(dollar_value)', 'dollar_spent')
                                       .withColumnRenamed('count(order_id)', 'no_orders'))


In [46]:
sales_by_region

state,postcode,order_datetime,dollar_spent,no_orders
NSW,2212,2021-07-21,1799.2035091441462,7
VIC,3194,2021-04-03,658.0240558265559,8
VIC,3123,2021-03-10,437.83024056803663,4
NSW,2472,2021-07-23,1339.0905111892507,14
NSW,2198,2021-06-30,632.2719516638596,9
NSW,2198,2021-07-29,236.9399442110043,4
VIC,3610,2021-06-01,2531.7924958543076,12
VIC,3053,2021-03-20,867.1311793948004,7
VIC,3631,2021-07-28,1224.3308918097925,8
QLD,4001,2021-05-16,659.0746049473196,9


In [ ]:
# Download data
sales_by_region.write.parquet("../curated/sales_by_region.parquet")